In [289]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

In [290]:
N = 4
m = gp.Model()

cap = 6.4
max_charge = cap/3
max_soe = cap*1.0
min_soe = cap*0.1
eta = [0.95]*N

action = 3.5
mu = [0.15, 0.75, 0.09, 0.01]
init_soe = [cap*0.2, cap*0.8, cap*0.3, cap*0.5] # np.array 사용하면 indexing할 때 조금 더러워짐.

#====================================================================================

charge = m.addVars(N, lb=0, vtype=GRB.CONTINUOUS, ub=max_charge, name='charge')
discharge = m.addVars(N, lb=0, vtype=GRB.CONTINUOUS, ub=max_charge, name='discharge')
self_ch = m.addVars(N, lb=0, vtype=GRB.CONTINUOUS, name='self_ch')
self_dis = m.addVars(N, lb=0, vtype=GRB.CONTINUOUS, name='self_dis')
ext_ch = m.addVars(N, lb=0, vtype=GRB.CONTINUOUS, name='ext_ch')
ext_dis = m.addVars(N, lb=0, vtype=GRB.CONTINUOUS, name='ext_dis')
soe = m.addVars(N, lb=min_soe, ub=max_soe, vtype=GRB.CONTINUOUS, name='soe')

diag = np.eye(N,N)

In [291]:
load = [2.4, 3.2, 4.2, 3.3]
pv = [4.8, 5.5, 5.3, 5.6]

In [292]:
for i in range(N):
    m.addConstr(  charge[i] == self_ch[i] + gp.quicksum(ext_ch[j] for j in range(N) if j != i)  ) # Net_Charge 구하기.
    m.addConstr(  discharge[i] == self_dis[i] + gp.quicksum(ext_dis[j] for j in range(N) if j != i)  ) # Net_Discharge 구하기.    
    m.addConstr(  soe[i] == init_soe[i] + eta[i]*charge[i] - (1/eta[i])*discharge[i]  ) # 에너지 충방전 과정.

m.addConstr( gp.quicksum(charge[i] - discharge[i] for i in range(N)) <= action ) # Action 값보다는 작아야.

'''
m.setObjective(gp.quicksum(mu[i]*(charge[i]-discharge[i]) for i in range(N)) 
                + 0.001*gp.quicksum(charge[i] + discharge[i] for i in range(N))
                , GRB.MAXIMIZE)
'''

m.setObjective( 15*gp.quicksum(load[i] - pv[i] + charge[i]-discharge[i] for i in range(N))
               + 0.001*gp.quicksum(charge[i] + discharge[i] for i in range(N))
               , GRB.MINIMIZE  )


m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 13 rows, 28 columns and 60 nonzeros
Model fingerprint: 0x48dd2921
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [6e-01, 6e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 11 rows and 24 columns
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.1285391e+02   0.000000e+00   0.000000e+00      0s
       0   -2.1285391e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -2.128539093e+02


In [293]:
for v in m.getVars():
    print(v.varName, np.round(v.x))

charge[0] 0.0
charge[1] 0.0
charge[2] 0.0
charge[3] 0.0
discharge[0] 1.0
discharge[1] 2.0
discharge[2] 1.0
discharge[3] 2.0
self_ch[0] 0.0
self_ch[1] 0.0
self_ch[2] 0.0
self_ch[3] 0.0
self_dis[0] 1.0
self_dis[1] 2.0
self_dis[2] 1.0
self_dis[3] 2.0
ext_ch[0] 0.0
ext_ch[1] 0.0
ext_ch[2] 0.0
ext_ch[3] 0.0
ext_dis[0] 0.0
ext_dis[1] 0.0
ext_dis[2] 0.0
ext_dis[3] 0.0
soe[0] 1.0
soe[1] 3.0
soe[2] 1.0
soe[3] 1.0
